In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\Archana\\Desktop\\test\\assignmen_solution\\research'

In [3]:
os.chdir('../')

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    

In [11]:
from src.ml_proj.utils.common import read_yaml, create_directories
from src.ml_proj.utils.common import save_json
from src.ml_proj.constants import *

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config1 = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        config2 = self.config.mlflow_info

        os.environ['MLFLOW_TRACKING_URI'] = config2.tracking_uri
        os.environ['MLFLOW_TRACKING_USERNAME'] = config2.tracking_username
        os.environ['MLFLOW_TRACKING_PASSWORD'] = config2.tracking_password
        create_directories([config1.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config1.root_dir,
            test_data_path=config1.test_data_path,
            model_path = config1.model_path,
            all_params=params,
            metric_file_name = config1.metric_file_name,
            target_column = schema.name,
            mlflow_uri= config2.tracking_uri,
           
        )

        return model_evaluation_config

In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
import joblib

In [14]:
import json
from urllib.parse import urlparse

In [25]:
%pip install mlflow

  Using cached alembic-1.15.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached matplotlib-3.10.1-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached sqlalchemy-2.0.39-cp310-cp310-win_amd64.whl.metadata (9.9 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached databricks_sdk-0.47.0-py3-none-any.whl.metadata (38 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_sdk-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached protobuf-5.29.4-cp3

In [26]:
import mlflow
import mlflow.sklearn

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_experiment("First Experiment")
        
        mlflow.set_tracking_uri(uri=self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            accuracy = accuracy_score(test_y, predicted_qualities)
            save_json(path=Path(self.config.metric_file_name), data=scores)
            

            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("accuracy",accuracy)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegressionModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    

In [21]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            
        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        print("Accuracy : ",accuracy_score(test_y, predicted_qualities))
        print("Scores: ",scores)
        save_json(path=Path(self.config.metric_file_name), data=scores)
            

            
            
      

    

In [22]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    
except Exception as e:
    raise e

[2025-03-27 12:58:55,942: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-27 12:58:55,946: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-27 12:58:55,952: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-27 12:58:55,954: INFO: common: created directory at: artifacts]
[2025-03-27 12:58:55,958: INFO: common: created directory at: artifacts/model_evaluation]
Accuracy :  0.8328
Scores:  {'rmse': np.float64(0.4089009660052174), 'mae': 0.1672, 'r2': 0.2956032837694188}
[2025-03-27 12:58:55,988: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
